#Member-1: 22MCS1018 (Harshad Katariya)
#Member-2: 22MCS1005 (Shubham Jadhav)

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np

In [ ]:
!pip install python-louvain


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/social_network/project/users.csv')

In [ ]:
G = nx.Graph()

In [ ]:
df.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset
0,3610511,Davide Dellacasa,braddd,20370,5470,2385,145,52,Fri Apr 06 10:58:22 +0000 2007,http://braddd.tumblr.com,...,FFF7CC,http://a0.twimg.com/profile_background_images/...,BADFCD,FF0000,3600.0,NaN,NaN,Founder of http://www.screenweek.it & http://w...,2015-02-14 10:54:49,E13
1,5656162,Simone Economo,eKoeS,3131,506,381,9,40,Mon Apr 30 15:08:42 +0000 2007,http://www.lineheight.net/,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,3600.0,NaN,NaN,BSc degree (cum laude) in Computer Engineering...,2015-02-14 10:54:49,E13
2,5682702,tacone,tacone_,4024,264,87,323,16,Tue May 01 11:53:40 +0000 2007,http://t.co/LKrl1dZE,...,000000,http://a0.twimg.com/profile_background_images/...,1A1B1F,2FC2EF,3600.0,NaN,NaN,Cogito ergo bestemmio.,2015-02-14 10:54:49,E13
3,6067292,alesaura,alesstar,40586,640,622,1118,32,Tue May 15 16:55:16 +0000 2007,http://alesstar.wordpress.com/,...,95E8EC,http://a0.twimg.com/images/themes/theme4/bg.gif,0099B9,0099B9,3600.0,NaN,NaN,"Se la vita ti dà sarde, scapocciale!",2015-02-14 10:54:49,E13
4,6015122,Angelo,PerDiletto,2016,62,64,13,0,Sun May 13 19:52:00 +0000 2007,http://www.flickr.com/per_diletto,...,F6F6F6,http://a0.twimg.com/images/themes/theme18/bg.gif,ACDED6,038543,3600.0,NaN,NaN,Je me souviens,2015-02-14 10:54:49,E13


In [ ]:
#Isolation Forest

from sklearn.ensemble import IsolationForest

# Select relevant features for anomaly detection
features = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count']

# Create a subset of the data with only the selected features
data_subset = df[features]

# Train the Isolation Forest model
model = IsolationForest(contamination=0.01)  # Adjust the contamination parameter as needed
model.fit(data_subset)

# Predict outliers/anomalies
outliers = model.predict(data_subset)
potential_fake_accounts = df[outliers == -1]['id'].tolist()

print("Potential Fake Accounts (Isolation Forest):")
print(potential_fake_accounts)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Potential Fake Accounts (Isolation Forest):
[15271419, 21102000, 56326516, 77528520, 110273768, 178507135, 215216086, 401502696, 420995289, 425263052, 452828946, 632742781, 120720544, 327900613, 522582939]


In [ ]:
#Local Outlier Factor (LOF):
from sklearn.neighbors import LocalOutlierFactor

# Select relevant features for anomaly detection
features = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count']

# Create a subset of the data with only the selected features
data_subset = df[features]

# Train the LOF model
model = LocalOutlierFactor(contamination=0.01)  # Adjust the contamination parameter as needed
outliers = model.fit_predict(data_subset)

# Extract potential fake accounts
potential_fake_accounts = df[outliers == -1]['id'].tolist()

print("Potential Fake Accounts (LOF):")
print(potential_fake_accounts)


Potential Fake Accounts (LOF):
[7046912, 15271419, 38002859, 64967572, 92403540, 104566770, 215216086, 399572723, 425380217, 523331583, 632742781, 706819428, 327900613, 522582939, 835342567]


In [ ]:
#Degree Centrality:
import networkx as nx

# Create a directed graph from the dataset
G = nx.from_pandas_edgelist(df, 'id', 'friends_count', create_using=nx.DiGraph())

# Compute the degree centrality for each node
degree_centrality = nx.degree_centrality(G)

# Identify nodes with high degree centrality (potential fake accounts)
potential_fake_accounts_d = [node for node, centrality in degree_centrality.items() if centrality > 0.0005]

print("Potential Fake Accounts (Degree Centrality):")
print(potential_fake_accounts_d)


Potential Fake Accounts (Degree Centrality):
[381, 87, 64, 179, 168, 218, 338, 203, 78, 571, 181, 371, 305, 146, 110, 276, 323, 109, 259, 131, 121, 267, 299, 407, 1846, 285, 174, 36, 80, 268, 257, 66, 178, 154, 295, 210, 239, 1449, 42, 301, 318, 205, 431, 122, 49, 73, 20, 460, 194, 390, 70, 463, 549, 112, 115, 96, 302, 202, 366, 152, 227, 336, 427, 138, 241, 173, 548, 237, 405, 380, 650, 290, 160, 185, 99, 226, 215, 454, 192, 1988, 162, 187, 281, 76, 75, 723, 770, 104, 53, 171, 18, 183, 111, 105, 250, 12, 536, 620, 282, 164, 161, 356, 546, 275, 139, 429, 2000, 156, 63, 283, 158, 198, 868, 314, 95, 100, 216, 320, 308, 364, 414, 68, 1172, 60, 220, 34, 126, 153, 77, 186, 217, 28, 213, 125, 40, 401, 81, 261, 193, 182, 271, 253, 255, 56, 135, 270, 348, 258, 190, 91, 59, 487, 41, 776, 533, 29, 626, 166, 102, 287, 232, 561, 369, 142, 286, 117, 310, 48, 243, 235, 85, 228, 627, 149, 165, 118, 448, 219, 175, 195, 400, 98, 209, 700, 554, 58, 94, 93, 428, 410, 469, 337, 341, 123, 383, 321, 92, 133

In [ ]:
print(degree_centrality)

{3610511: 0.0004597701149425287, 2385: 0.0004597701149425287, 5656162: 0.0004597701149425287, 381: 0.0009195402298850574, 5682702: 0.0004597701149425287, 87: 0.0022988505747126436, 6067292: 0.0004597701149425287, 622: 0.0004597701149425287, 6015122: 0.0004597701149425287, 64: 0.0018390804597701149, 6140012: 0.0004597701149425287, 179: 0.0022988505747126436, 6134312: 0.0004597701149425287, 168: 0.0036781609195402297, 6684602: 0.0004597701149425287, 1770: 0.0004597701149425287, 7046912: 0.0004597701149425287, 958: 0.0004597701149425287, 7470952: 0.0004597701149425287, 712: 0.0004597701149425287, 8072492: 0.0004597701149425287, 218: 0.0018390804597701149, 8291932: 0.0004597701149425287, 1177: 0.0004597701149425287, 8858022: 0.0004597701149425287, 338: 0.0018390804597701149, 8927532: 0.0004597701149425287, 203: 0.003218390804597701, 8933252: 0.0004597701149425287, 1930: 0.0004597701149425287, 9351052: 0.0004597701149425287, 78: 0.0018390804597701149, 9564632: 0.0004597701149425287, 918: 0.

In [ ]:
#PageRank:
import networkx as nx

# Create a directed graph from the dataset
G = nx.from_pandas_edgelist(df, 'id', 'friends_count', create_using=nx.DiGraph())

# Compute the PageRank score for each node
pagerank_scores = nx.pagerank(G)

# Identify nodes with high PageRank scores (potential fake accounts)
potential_fake_accounts_p = [node for node, score in pagerank_scores.items() if score > 0.0005]

print("Potential Fake Accounts (PageRank):")
print(potential_fake_accounts_p)


Potential Fake Accounts (PageRank):
[2385, 381, 87, 622, 64, 179, 168, 1770, 958, 712, 218, 1177, 338, 203, 1930, 78, 918, 891, 571, 181, 371, 305, 1039, 146, 110, 647, 1096, 276, 419, 3873, 1125, 323, 873, 109, 259, 131, 1047, 196, 755, 121, 951, 267, 2003, 1505, 1446, 299, 407, 1846, 285, 174, 36, 80, 268, 257, 66, 178, 154, 295, 1089, 210, 239, 1449, 42, 330, 301, 318, 1743, 205, 1642, 793, 489, 1513, 431, 122, 49, 73, 20, 191, 460, 194, 390, 473, 70, 463, 502, 549, 112, 433, 406, 115, 1153, 1196, 656, 96, 1376, 302, 202, 352, 366, 347, 152, 688, 934, 227, 336, 427, 138, 697, 1273, 241, 173, 768, 136, 345, 548, 237, 405, 380, 650, 290, 160, 1974, 185, 99, 226, 1221, 635, 215, 590, 1598, 454, 192, 5530, 1988, 162, 187, 281, 76, 75, 723, 552, 418, 770, 104, 53, 171, 18, 802, 183, 111, 105, 199, 250, 12, 536, 620, 282, 164, 161, 356, 546, 275, 262, 774, 139, 324, 429, 2000, 1771, 667, 443, 156, 392, 63, 283, 158, 198, 761, 601, 860, 12773, 868, 314, 95, 100, 216, 320, 1853, 308, 364, 5

In [ ]:
print(pagerank_scores)

{3610511: 0.0002913702230394513, 2385: 0.0005384859520510283, 5656162: 0.0002913702230394513, 381: 0.0007856016810626052, 5682702: 0.0002913702230394513, 87: 0.0015269488680973362, 6067292: 0.0002913702230394513, 622: 0.0005384859520510283, 6015122: 0.0002913702230394513, 64: 0.0012798331390857592, 6140012: 0.0002913702230394513, 179: 0.0015269488680973362, 6134312: 0.0002913702230394513, 168: 0.0022682960551320673, 6684602: 0.0002913702230394513, 1770: 0.0005384859520510283, 7046912: 0.0002913702230394513, 958: 0.0005384859520510283, 7470952: 0.0002913702230394513, 712: 0.0005384859520510283, 8072492: 0.0002913702230394513, 218: 0.0012798331390857592, 8291932: 0.0002913702230394513, 1177: 0.0005384859520510283, 8858022: 0.0002913702230394513, 338: 0.0012798331390857592, 8927532: 0.0002913702230394513, 203: 0.00202118032612049, 8933252: 0.0002913702230394513, 1930: 0.0005384859520510283, 9351052: 0.0002913702230394513, 78: 0.0012798331390857592, 9564632: 0.0002913702230394513, 918: 0.0

In [ ]:
potential_fake_accounts_iforest = [3610511, 15271419, 21102000, 56326516, 77528520, 92403540, 110273768, 215216086,338, 420995289, 425263052, 632742781, 120720544, 327900613,571, 181, 371, 305, 146, 110, 276, 323, 109, 259, 131, 121, 267, 299, 407, 1846, 285, 174, 522582939, 558489220, 425380217,]
potential_fake_accounts_lof = [7046912, 15271419, 38002859, 64967572, 92403540,338, 104566770, 110273768, 215216086, 399572723, 425380217, 523331583,420995289, 632742781,571, 181, 371, 305, 146, 110, 276, 323, 109, 259, 131,56326516, 77528520, 92403540, 110273768, 215216086, 121, 267, 299, 407, 1846, 285, 174, 706819428, 327900613, 522582939, 835342567]
potential_fake_accounts_degree = [381, 87, 64, 179, 168, 218, 338, 203, 78, 571, 181, 371, 305, 146, 110, 276, 323, 109, 259, 131, 121, 267, 299, 407, 1846, 285, 174, 36, 80, 268,338, 257, 66, 178, 154, 295, 210, 239, 1449, 42, 301, 318,56326516, 77528520, 92403540, 110273768, 215216086, 205, 431, 122, 49, 73, 20, 460, 194, 390, 70, 463, 549, 112, 115, 96, 302, 202, 366, 152, 227, 336, 427, 138, 241, 173, 548, 237, 405, 380, 650, 290,420995289, 160, 185, 99, 226, 215, 454, 192, 1988, 162, 187, 281, 76, 75, 723, 770, 104, 53, 171, 18, 183, 111, 105, 250, 12, 536, 620, 282, 164, 161, 356, 546, 275, 139, 429, 2000, 156, 63, 283, 158, 198, 868, 314, 95, 100, 216, 320, 308, 364, 414, 68, 1172, 60, 220, 34, 126, 153, 77, 186, 217, 28, 213, 125, 40, 401, 81, 261, 193, 182, 271, 253, 255, 56, 135, 270, 348, 258, 190, 91, 59, 487, 41, 776, 533, 29, 626, 166, 102, 287, 232, 561, 369, 142, 286, 117, 310, 48, 243, 235, 85, 228, 627, 149, 165, 118, 448, 219, 425380217, 175, 195, 400, 98, 209, 700, 554, 58, 94, 93, 428, 410, 469, 337, 341, 123, 383, 321, 92, 133, 236, 361, 176, 206, 332, 170, 37, 145, 128, 289, 886, 273, 38, 899, 1428, 148, 256, 89, 528, 163, 646, 157, 54, 662, 360, 229, 230, 962, 137, 346, 50, 1994, 90, 22, 74, 143, 904, 106, 222, 599, 516, 249, 201, 1999, 159, 745, 522, 358, 396, 413, 97, 79, 71, 342, 204, 72, 1027, 88, 51, 362, 101, 55, 200, 46, 388, 272, 180, 114, 242, 351, 84, 23, 279, 21, 329, 327, 269, 45, 140, 389, 335, 300, 233, 816, 31, 266, 384, 147, 278, 172, 35, 141, 284, 495, 86, 127, 169, 403, 150, 197, 107, 67, 65, 245, 120, 189, 108, 480, 30, 129, 221, 57, 69, 309, 725, 167, 280, 322, 52, 659, 188, 373, 43, 207, 124, 83, 32, 4]
potential_fake_accounts_pagerank = [2385, 381, 87, 622, 64, 179, 168, 1770, 958, 712, 218, 1177, 338, 203, 1930, 78,571, 181, 371, 305, 146, 110, 276, 323, 109, 259, 131, 121, 267, 299, 407, 1846, 285, 174, 918, 891, 571, 181, 371, 305, 1039, 146, 110, 647, 1096, 276, 419, 3873, 1125, 323, 873, 109, 259, 131, 1047, 196, 755, 121, 951, 267, 2003, 1505, 1446, 299, 407,56326516, 77528520, 92403540, 110273768, 215216086, 1846, 110273768,285, 174, 36, 80, 268, 257, 66, 178, 154, 295, 1089, 210, 239, 1449, 42, 330, 301, 318, 1743, 205, 1642, 793, 489, 1513, 431, 122, 49, 73, 20, 191, 460, 194, 390, 473, 70, 463, 502, 549, 112, 433,420995289, 406, 115, 1153, 1196, 656, 96, 1376, 302, 202, 352, 366, 347, 152, 688, 934, 227, 336, 427, 138, 697, 1273, 241, 173, 768, 136, 345, 548, 237, 405, 380, 650, 290, 160, 1974, 185, 99, 226, 1221, 635, 215, 590, 1598, 454, 192, 5530, 1988,110273768, 162, 187, 281, 76, 75, 723, 552, 418, 770, 104, 53, 171, 18, 802, 183, 111, 105, 199, 250, 12, 536, 620, 282, 164, 161, 356, 546, 275, 262, 774, 139, 324, 429, 2000, 1771, 667, 443, 156, 392, 63, 283, 158, 198, 761, 601, 860, 12773, 868, 314, 95, 100, 216, 320, 1853, 308, 364, 506, 1159, 414, 68, 1172, 60, 220, 1636, 34, 126, 153, 77, 611, 186, 217, 294, 368, 28, 213, 125, 40, 401, 81, 261, 193, 182, 425380217, 271, 253, 255, 1021, 377, 565, 521, 254, 543, 619, 56, 1581, 135, 270, 826, 348, 840, 3032, 258, 14, 2743, 1995, 190, 781, 91, 59, 452, 1898, 461, 500, 625, 487, 937, 41, 776, 538, 533, 423, 1416, 437, 29, 626, 1516, 1442, 471, 166, 102, 706, 287, 411, 232, 561, 369, 142, 286, 117, 310, 938, 48, 243, 235, 85, 228, 334, 627, 149, 165, 118, 448, 219, 175, 195, 956, 251, 400, 252, 445, 588, 98, 209, 700, 554, 58, 94, 93, 428, 970, 410, 2002, 469, 1686, 184, 337, 341, 1609, 1020, 123, 560, 383, 321, 92, 133, 236, 361, 176, 206, 332, 1968, 170, 37, 145, 128, 879, 693, 289, 1625, 886, 273, 38, 834, 899, 547, 1428, 787, 148, 256, 5335, 89, 660, 1163, 528, 993, 163, 646, 1529, 350, 157, 54, 662, 1978, 360, 340, 229, 230, 892, 417, 962, 137, 1433, 1462, 346, 488, 316, 50, 1994, 90, 22, 74, 143, 904, 490, 106, 222, 599, 516, 370, 701, 801, 249, 201, 778, 313, 1999, 159, 1491, 745, 522, 358, 263, 396, 413, 523, 97, 79, 1524, 71, 342, 315, 204, 661, 582, 72, 1953, 1027, 1409, 88, 441, 51, 362, 1158, 101, 55, 344, 685, 491, 200, 1828, 46, 653, 388, 550, 248, 211, 272, 180, 114, 242, 1435, 613, 351, 84, 678, 317, 1627, 578, 494, 23, 614, 279, 1012, 21, 155, 1225, 925, 632, 648, 329, 297, 643, 327, 269, 45, 355, 140, 775, 389, 535, 307, 457, 1108, 1275, 525, 304, 335, 300, 233, 497, 436, 1996, 816, 967, 31, 266, 384, 147, 212, 357, 757, 278, 731, 172, 35, 623, 141, 134, 722, 284, 495, 86, 349, 909, 475, 681, 127, 1474, 82, 997, 1320, 325, 509, 169, 240, 1504, 265, 403, 150, 197, 1042, 1059, 507, 107, 995, 2001, 486, 67, 513, 343, 2744, 65, 245, 474, 120, 1970, 1328, 25, 189, 562, 214, 103, 404, 803, 151, 130, 108, 1532, 376, 17, 33, 932, 821, 795, 480, 30, 576, 666, 570, 455, 129, 47, 472, 558, 221, 353, 113, 57, 69, 1281, 309, 1991, 1109, 725, 397, 824, 167, 499, 1441, 1998, 244, 728, 1908, 435, 936, 1340, 602, 378, 280, 322, 785, 62, 52, 659, 188, 373, 524, 27, 288, 43, 540, 992, 274, 1842, 207, 6, 808, 402, 555, 1856, 1019, 912, 408, 391, 1987, 671, 707, 606, 608, 1876, 759, 715, 399, 1088, 119, 991, 610, 1697, 177, 1200, 116, 637, 1249, 246, 1015, 124, 83, 663, 1478, 567, 292, 10, 503, 1304, 359, 440, 1993, 512, 705, 1092, 1224, 911, 144, 628, 1440, 319, 412, 298, 638, 296, 736, 1187, 32, 1391, 901, 1448, 3313, 797, 880, 260, 425, 306, 24, 498, 446, 208, 1653, 580, 1242, 885, 5835, 1006, 831, 4071, 2406, 996, 4120, 382, 16, 874, 792, 132, 618, 1527, 1558, 1650, 44, 530, 19, 11, 0, 4]

In [ ]:
G = nx.from_pandas_edgelist(df, 'id', 'friends_count')

In [ ]:
#Combine result

In [ ]:
def detect_fake_accounts(results_list, threshold):
    # Create an empty dictionary to store vote counts
    vote_counts = {}

    # Iterate over the individual results
    for result in results_list:
        for account_id in result:
            if account_id in vote_counts:
                vote_counts[account_id] += 1
            else:
                vote_counts[account_id] = 1

    # Identify potential fake accounts based on the vote counts
    potential_fake_accounts = [account_id for account_id, votes in vote_counts.items() if votes >= threshold]

    return potential_fake_accounts


# Combine results and set a threshold
combined_results = [potential_fake_accounts_iforest,
                    potential_fake_accounts_lof,
                    potential_fake_accounts_degree,
                    potential_fake_accounts_pagerank]


threshold = 3

# Detect potential fake accounts
potential_fake_accounts = detect_fake_accounts(combined_results, threshold)

print("Potential Fake Accounts:")
print(potential_fake_accounts)


Potential Fake Accounts:
[56326516, 77528520, 92403540, 110273768, 215216086, 338, 420995289, 571, 181, 371, 305, 146, 110, 276, 323, 109, 259, 131, 121, 267, 299, 407, 1846, 285, 174, 425380217]
